In [1]:
!pip install pdfplumber
!pip install openai==0.28.1
!pip install tiktoken
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.17
    Uninstalling langchain-core-0.3.17:
      Successfully uninstalled langchain-core-0.3.17
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-spli

In [2]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import warnings
import os
warnings.filterwarnings("ignore")
import pdfplumber

In [14]:
api_key = ''

In [15]:
openai.api_key = api_key

In [4]:
def extract_tables_from_pdf(pdf_path):
  tables = []
  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
      tables.extend(page.extract_tables())
  return tables

In [5]:
tables = extract_tables_from_pdf("/content/Dentist.pdf")

In [6]:
dataframes = []
dataframed = None
for table in tables:
  dataframed = pd.DataFrame(table[1:], columns=table[0])
  dataframes.append(dataframed)

In [7]:
dataframed['combined'] = dataframed.apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [8]:
dataframed

,CLINIC NAME,CLINIC HOURS,CONTACT NO,STREET,BRGY,MUNICIPAL,PROVINCE,REGION,combined
0,ALIP DENTAL CLINIC,M-S 9:00AM-5:00PM BY APPOINTMENT,"(074) 448-8341, (0917) 627-2610","B108 JUNIPER BLDG.,",GEN. LUNA RD,BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION,ALIP DENTAL CLINIC M-S 9:00AM-5:00PM BY APPOIN...
1,SMILE DENTAL CLINIC,M-S 9:00AM-5:00PM,(074) 442-9028,UNIT 3-B / 5 & 6 PUSO,"NSGES BSIAOGNU RIOO,AD",BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION,SMILE DENTAL CLINIC M-S 9:00AM-5:00PM (074) 44...
2,THE DENTAL STUDIO BY: DR. KRISHA SY,M-S 9:00AM-6:00PM,(0917) 506-9031,2ND FLR. ECCO BLDG.,"ASSUMPTION ROAD,",BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION,THE DENTAL STUDIO BY: DR. KRISHA SY M-S 9:00AM...
3,GENOVEZA DENTAL CLINIC,M-S 9:00AM-6:00PM; SUN (BY APPOINTMENT),(074) 423-0947,GF 104 BALISONG BLDG,"., ASSUMPTION ROAD.,",BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION,GENOVEZA DENTAL CLINIC M-S 9:00AM-6:00PM; SUN ...
4,DR. ANNA LEAH TORRES-SISON DENTAL CLINIC,M-S 10:AM-5:00PM BY APPOINTMENT,"(074) 442-4040, (0917) 560-4562","RM. A1, 3/F DOÑA ANIT","A#6 B6L DSEGS.S 2IO,N ROAD",BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION,DR. ANNA LEAH TORRES-SISON DENTAL CLINIC M-S 1...
...,...,...,...,...,...,...,...,...,...
161,GDH DENTAL CLINIC,M-S 10:00AM-6:00PM BY APPOINTMENT,(02) 410-2257,10-A STO. NINO ST.,"BRGY. SAN ANTONIO, S",AQNU FERZAONNC CISICTOY DEL M,ODNITSETRICT 2,NCR - NATIONAL CAPITAL REGION,GDH DENTAL CLINIC M-S 10:00AM-6:00PM BY APPOIN...
162,DENTAL CARE CORNER,M-S 10:00AM-7:00PM,"(02) 928-0256, (02) 957-4416, (0906) 305-6031","2/F UNIT A, 56 VISAYAS",ABVREGNYU. VEASRA,QUEZON CITY,DISTRICT 2,NCR - NATIONAL CAPITAL REGION,DENTAL CARE CORNER M-S 10:00AM-7:00PM (02) 928...
163,DR. FLORENCE A. DE VENECIA DENTAL CLINIC - UCP...,M-S 9:00AM-5:00PM BY APPOINTMENT,"(02) 736-7142, (02) 924-2369, (02) 990-2373, (...",RM 403 UCPI (KALAYAA,"ND)I,L IMMAAGNI,NOO ST.,",QUEZON CITY,DISTRICT 2,NCR - NATIONAL CAPITAL REGION,DR. FLORENCE A. DE VENECIA DENTAL CLINIC - UCP...
164,BUHAIN DENTAL CLINIC,M-S 9:00AM-5:00PM STRICTLY BY APPOINTMENT,"(02) 434-8181, (02) 435-3556, (0927) 498-5239",UNIT 102 EAGLE STAR C,OLONYDOOL.A 2 H5 EFIAGBHIATSN DELA,QRUOESZAO SNT. CITY,DISTRICT 2,NCR - NATIONAL CAPITAL REGION,BUHAIN DENTAL CLINIC M-S 9:00AM-5:00PM STRICTL...


In [16]:
docs = dataframed['combined'].tolist()
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in docs]
embedding_dim = len(embeddings[0])
embeddings_np = np.array(embeddings).astype('float32')

index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)

In [21]:
prompt = 'Where is the alex Dental clinic'
#SYSTEM_PROMPT = ''
#struc = [{"role": "system", "content": SYSTEM_PROMPT}]
struc = [{"role": "system", "content": 'You are a helpful assistant'}]
query_embed = get_embedding(prompt, engine='text-embedding-3-small')
query_embed_np = np.array([query_embed]).astype('float32')
_, indices = index.search(query_embed_np, 2)
retrieved_docs = [docs[i] for i in indices[0]]
context = ' '.join(retrieved_docs)

structured_prompt = f"Context:\n{context}\n\nQuery:\n{prompt}\n\nResponse:"
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struc + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struc.append({"role": "user", "content": prompt})
response = chat.choices[0].message.content

In [22]:
print(response)

The provided context does not mention an "Alex Dental Clinic." It only includes information about Alonzo Dental Clinic and Olayvar Dental Clinic. If you are looking for a specific dental clinic named Alex, you may need to provide more information or check other sources.


In [23]:
print(structured_prompt)

Context:
ALONZO DENTAL CLINIC M-S 10:00AM-5:00PM BY APPOINTMENT (02) 646-2015, (02) 668-1076 OLD #35 NEW #26 W. P ASZT AS.T E.LENA MARIKINA CITY DISTRICT 2 NCR - NATIONAL CAPITAL REGION OLAYVAR DENTAL CLINIC M-S 9:00AM-5:00PM BY APPOINTMENT (02) 723-0101 [5810] RM. 907 NORTH TOWE RE .C RHOBDCRIGUEZ, ST. LUK EQ'SU MEZEODINC ACLIT CYENTER, DISTRICT 2 NCR - NATIONAL CAPITAL REGION

Query:
Where is the alex Dental clinic

Response:
